In [103]:
import pandas as pd
import re
import numpy as np

import json

In [62]:
df_listing = pd.read_csv('listings.csv')

## Price in Calender

In [2]:
df = pd.read_csv('calendar.csv')

In [3]:
df.describe()

,listing_id,minimum_nights,maximum_nights
count,7.457698e+06,7.457218e+06,7.457218e+06
mean,2.332012e+07,8.051325e+00,2.103489e+05
std,1.360332e+07,3.038445e+01,2.121709e+07
min,1.944000e+03,1.000000e+00,1.000000e+00
25%,1.162000e+07,2.000000e+00,3.000000e+01
50%,2.237011e+07,3.000000e+00,1.125000e+03
75%,3.595552e+07,5.000000e+00,1.125000e+03
max,4.519743e+07,1.124000e+03,2.147484e+09


In [4]:
df.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,186851,2020-09-01,f,$33.00,$33.00,62.0,300.0
1,1944,2020-08-31,f,$20.00,$20.00,60.0,120.0
2,1944,2020-09-01,f,$20.00,$20.00,60.0,120.0
3,1944,2020-09-02,f,$20.00,$20.00,60.0,120.0
4,1944,2020-09-03,f,$20.00,$20.00,60.0,120.0


In [6]:
datecol = pd.to_datetime(df['date'])
datecol

0         2020-09-01
1         2020-08-31
2         2020-09-01
3         2020-09-02
4         2020-09-03
             ...    
7457693   2021-08-26
7457694   2021-08-27
7457695   2021-08-28
7457696   2021-08-29
7457697   2021-08-30
Name: date, Length: 7457698, dtype: datetime64[ns]

In [8]:
datecol.min(), datecol.max()

(Timestamp('2020-08-30 00:00:00'), Timestamp('2021-08-31 00:00:00'))

In [9]:
df['date'] = datecol

In [10]:
df.groupby('date').count()

,listing_id,available,price,adjusted_price,minimum_nights,maximum_nights
date,,,,,,
2020-08-30,2907,2907,2907,2907,2817,2817
2020-08-31,12451,12451,12451,12451,12147,12147
2020-09-01,20431,20431,20431,20431,20431,20431
2020-09-02,20431,20431,20431,20431,20431,20431
2020-09-03,20431,20431,20431,20431,20431,20431
...,...,...,...,...,...,...
2021-08-27,20430,20430,20430,20430,20427,20427
2021-08-28,20430,20430,20430,20430,20427,20427
2021-08-29,20431,20431,20431,20431,20430,20430


In [15]:
# only preserve listings with complete record from 2020-09-01 to 2021-08-29: 363 days
listing_date_cnt = df.groupby('listing_id').agg({'date': ['max', 'min', 'count']})
listing_date_cnt = listing_date_cnt[(listing_date_cnt[('date', 'max')] >= pd.to_datetime('2021-08-29')) & \
                 (listing_date_cnt[('date', 'min')] <= pd.to_datetime('2020-09-01')) & \
                 (listing_date_cnt['date', 'count'] >= 363)]
listing_date_cnt

date                 
                  max        min count
listing_id                            
1944       2021-08-30 2020-08-31   365
3176       2021-08-30 2020-08-31   365
3309       2021-08-31 2020-09-01   365
7071       2021-08-30 2020-08-31   365
9991       2021-08-30 2020-08-31   365
...               ...        ...   ...
45188959   2021-08-31 2020-09-01   365
45189970   2021-08-31 2020-09-01   365
45190519   2021-08-31 2020-09-01   365
45195914   2021-08-31 2020-09-01   365
45197431   2021-08-31 2020-09-01   365

[20430 rows x 3 columns]

In [16]:
df_f = df[(df['listing_id'].isin(listing_date_cnt.index))]
df_f = df_f[(df_f['date'] <= '2021-08-29') & (df_f['date'] >= '2020-09-01')]
df_f

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,186851,2020-09-01,f,$33.00,$33.00,62.0,300.0
2,1944,2020-09-01,f,$20.00,$20.00,60.0,120.0
3,1944,2020-09-02,f,$20.00,$20.00,60.0,120.0
4,1944,2020-09-03,f,$20.00,$20.00,60.0,120.0
5,1944,2020-09-04,f,$20.00,$20.00,60.0,120.0
...,...,...,...,...,...,...,...
7457692,45133552,2021-08-25,f,$26.00,$26.00,3.0,60.0
7457693,45133552,2021-08-26,f,$26.00,$26.00,3.0,60.0
7457694,45133552,2021-08-27,f,$26.00,$26.00,3.0,60.0
7457695,45133552,2021-08-28,f,$26.00,$26.00,3.0,60.0


In [20]:
vc_date = df_f['date'].value_counts()
vc_date

2021-04-14    20430
2020-09-20    20430
2020-09-07    20430
2021-02-19    20430
2021-04-02    20430
              ...  
2021-04-21    20430
2021-05-12    20430
2021-07-10    20430
2021-01-01    20430
2021-04-09    20430
Name: date, Length: 363, dtype: int64

In [22]:
df_f.isna().sum()

listing_id         0
date               0
available          0
price              0
adjusted_price     0
minimum_nights    85
maximum_nights    85
dtype: int64

In [26]:
df_listing.columns 
# host_is_superhost, neighbourhood_group_cleansed, room_type, beds, availability_365
# review_scores_rating

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_upd

In [45]:
df_listing.review_scores_value.isna().sum()

4313

In [85]:
df_listing_for_price = df_listing[['id', 'host_is_superhost', 'neighbourhood_group_cleansed', 
                                   'room_type', 'beds', 'availability_365', 'review_scores_rating']]
df_listing_for_price.isna().sum()

id                                 0
host_is_superhost                 12
neighbourhood_group_cleansed       0
room_type                          0
beds                             236
availability_365                   0
review_scores_rating            4279
dtype: int64

In [86]:
df_listing_for_price = df_listing_for_price.dropna()

In [87]:
df_listing_for_price.review_scores_rating.value_counts()

100.0    5345
98.0     1297
97.0     1132
96.0     1088
95.0      921
93.0      917
90.0      795
99.0      791
94.0      642
80.0      635
92.0      474
91.0      365
87.0      279
89.0      240
88.0      238
85.0      127
86.0      114
84.0      106
60.0      102
83.0       95
73.0       45
70.0       41
82.0       32
20.0       29
81.0       29
75.0       27
40.0       25
77.0       22
76.0       22
78.0       17
67.0       13
72.0       11
79.0        7
50.0        6
74.0        6
71.0        6
65.0        5
68.0        4
47.0        2
64.0        2
53.0        2
56.0        1
57.0        1
66.0        1
63.0        1
55.0        1
Name: review_scores_rating, dtype: int64

In [88]:
# beds: to string, convert >= 7.0 to the same group
df_listing_for_price.loc[df_listing_for_price['beds'] > 7, 'beds'] = 7.0

In [101]:
df_listing_for_price['beds'] = pd.Series(df_listing_for_price.beds, dtype='string')

In [107]:
df_listing_for_price.loc[df_listing_for_price['beds'] == 7, 'beds'] = '7+'

In [108]:
df_listing_for_price.beds.value_counts()

1.0    9276
2.0    3886
3.0    1133
4.0     621
0.0     540
5.0     245
6.0     186
7+      174
Name: beds, dtype: int64

In [109]:
# bin availability 365
df_listing_for_price.availability_365 = pd.cut(df_listing_for_price.availability_365, 
                                               bins=[0, 50, 100, 150, 200, 250, 300, 400], 
       include_lowest=True, labels=['0-50', '51-100', '101-150', '151-200', '201-250', '251-300', '300+'])

In [110]:
# bin review_scores_rating
df_listing_for_price.review_scores_rating = pd.cut(df_listing_for_price.review_scores_rating, 
                                               bins=[0, 70, 80, 90, 100], 
       labels=['below 70', '71-80', '81-90', '91-100'])

In [111]:
df_listing_for_price.isna().sum()

id                              0
host_is_superhost               0
neighbourhood_group_cleansed    0
room_type                       0
beds                            0
availability_365                0
review_scores_rating            0
dtype: int64

In [112]:
df_price = df_listing_for_price.merge(df_f, left_on='id', right_on='listing_id')
df_price.head()

,id,host_is_superhost,neighbourhood_group_cleansed,room_type,beds,availability_365,review_scores_rating,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,1944,f,Mitte,Private room,1,251-300,81-90,1944,2020-09-01,f,$20.00,$20.00,60.0,120.0
1,1944,f,Mitte,Private room,1,251-300,81-90,1944,2020-09-02,f,$20.00,$20.00,60.0,120.0
2,1944,f,Mitte,Private room,1,251-300,81-90,1944,2020-09-03,f,$20.00,$20.00,60.0,120.0
3,1944,f,Mitte,Private room,1,251-300,81-90,1944,2020-09-04,f,$20.00,$20.00,60.0,120.0
4,1944,f,Mitte,Private room,1,251-300,81-90,1944,2020-09-05,f,$20.00,$20.00,60.0,120.0


In [120]:
pricecol = df_price.price.apply(lambda x: re.sub("[^0-9]", "", x[1:-3])).astype(int)

In [121]:
pricecol.describe()

count    5.829780e+06
mean     7.094809e+01
std      1.006422e+02
min      8.000000e+00
25%      3.500000e+01
50%      5.300000e+01
75%      8.000000e+01
max      6.632000e+03
Name: price, dtype: float64

In [128]:
df_price['price'] = pricecol

In [131]:
df_price.groupby('date').agg({'price': 'mean'})

,price
date,
2020-09-01,66.243898
2020-09-02,66.220299
2020-09-03,66.669054
2020-09-04,68.795081
2020-09-05,69.194022
...,...
2021-08-25,74.915816
2021-08-26,74.978456
2021-08-27,76.648007


In [132]:
df_price.to_csv('price.csv', index=False)

In [146]:
df_price_superhost = df_price[['host_is_superhost', 'date', 'price']]\
    .groupby(['host_is_superhost', 'date'], as_index=False).mean()
df_price_superhost.loc[df_price_superhost['host_is_superhost'] == 't', 'host_is_superhost'] = 'Yes'
df_price_superhost.loc[df_price_superhost['host_is_superhost'] == 'f', 'host_is_superhost'] = 'No'
df_price_superhost.to_csv('price_superhost.csv', index=False)

In [139]:
df_price['listing_id'].unique().shape

(16060,)

In [141]:
df_listing_for_price['id'].unique().shape

(16061,)

In [154]:
df_listing_for_price.head()

,id,host_is_superhost,neighbourhood_group_cleansed,room_type,beds,availability_365,review_scores_rating
0,1944,No,Mitte,Private room,1,251-300,81-90
1,3176,No,Pankow,Entire home/apt,2,300+,91-100
2,3309,No,Tempelhof - Schöneberg,Private room,1,201-250,81-90
3,7071,Yes,Pankow,Private room,2,0-50,91-100
4,9991,No,Pankow,Entire home/apt,7+,0-50,91-100


In [152]:
df_listing_for_price.loc[df_listing_for_price['host_is_superhost'] == 't', 'host_is_superhost'] = 'Yes'
df_listing_for_price.loc[df_listing_for_price['host_is_superhost'] == 'f', 'host_is_superhost'] = 'No'

In [153]:
df_listing_for_price.to_csv('price_listings.csv', index=False)

In [149]:
df_price_neighbourhood_group_cleansed = df_price[['neighbourhood_group_cleansed', 'date', 'price']]\
    .groupby(['neighbourhood_group_cleansed', 'date'], as_index=False).mean()
df_price_neighbourhood_group_cleansed.to_csv('price_neighborhood.csv', index=False)

In [150]:
df_price_room_type = df_price[['room_type', 'date', 'price']]\
    .groupby(['room_type', 'date'], as_index=False).mean()
df_price_room_type.to_csv('price_roomtype.csv', index=False)

In [151]:
df_price_beds = df_price[['beds', 'date', 'price']]\
    .groupby(['beds', 'date'], as_index=False).mean()
df_price_beds.to_csv('price_beds.csv', index=False)

In [155]:
# price and availability, price and review_scores_rating, beds
df_price_availability = df_price[['availability_365', 'date', 'price']]\
    .groupby(['availability_365', 'date'], as_index=False).mean()
df_price_availability.to_csv('price_availability.csv', index=False)

In [156]:
# price and availability, price and review_scores_rating, beds
df_price_review = df_price[['review_scores_rating', 'date', 'price']]\
    .groupby(['review_scores_rating', 'date'], as_index=False).mean()
df_price_review.to_csv('price_review.csv', index=False)

## listing, availbility

In [63]:
df_listing.head()

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,1944,https://www.airbnb.com/rooms/1944,20200830153825,2020-08-31,cafeheaven Pberg/Mitte,"Private, bright and friendly room. You'd be sh...",near all the trendy cafés and flea markets and...,https://a0.muscache.com/pictures/107924758/1f4...,2164,https://www.airbnb.com/users/show/2164,...,9.0,9.0,8.0,NaN,f,2,0,2,0,0.21
1,3176,https://www.airbnb.com/rooms/3176,20200830153825,2020-08-31,Fabulous Flat in great Location,This beautiful first floor apartment is situa...,The neighbourhood is famous for its variety of...,https://a0.muscache.com/pictures/243355/84afcf...,3718,https://www.airbnb.com/users/show/3718,...,9.0,10.0,9.0,NaN,f,1,1,0,0,1.07
2,3309,https://www.airbnb.com/rooms/3309,20200830153825,2020-09-01,BerlinSpot Schöneberg near KaDeWe,First of all:<br />I prefer short-notice booki...,"My flat is in the middle of West-Berlin, direc...",https://a0.muscache.com/pictures/29054294/b1fa...,4108,https://www.airbnb.com/users/show/4108,...,10.0,9.0,9.0,NaN,f,1,0,1,0,0.31
3,7071,https://www.airbnb.com/rooms/7071,20200830153825,2020-08-31,BrightRoom with sunny greenview!,Cozy and large room in the beautiful district ...,"Great neighborhood with plenty of Cafés, Baker...",https://a0.muscache.com/pictures/21278/32a1622...,17391,https://www.airbnb.com/users/show/17391,...,10.0,10.0,9.0,NaN,t,1,0,1,0,2.18
4,9991,https://www.airbnb.com/rooms/9991,20200830153825,2020-08-31,Geourgeous flat - outstanding views,4 bedroom with very large windows and outstand...,Prenzlauer Berg is an amazing neighbourhood wh...,https://a0.muscache.com/pictures/42799131/59c8...,33852,https://www.airbnb.com/users/show/33852,...,10.0,10.0,10.0,03/Z/RA/003410-18,f,1,1,0,0,0.13


In [64]:
df_listings_to_consider = pd.read_csv("price_listings.csv")[["id"]]

In [65]:
df_listing = df_listing.merge(df_listings_to_consider, left_on = "id", right_on = "id")

In [66]:
df_listing.shape

(16061, 74)

In [67]:
df_listing_sub = df_listing.sample(n=1000, random_state=42)

In [68]:
df_listing_sub.shape

(1000, 74)

In [69]:
df_listing_sub.to_csv("listing_sampled.csv", index=False)

In [70]:
df_listing_sub.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_upd

In [71]:
df_use = df_listing_sub[['price', 'review_scores_rating', 'availability_90', 'neighbourhood_group_cleansed', 'host_acceptance_rate',
                'host_response_time', 'host_identity_verified', 'host_has_profile_pic', 'number_of_reviews',
                'instant_bookable', 'host_is_superhost']]

In [72]:
df_use.isna().sum()

price                             0
review_scores_rating              0
availability_90                   0
neighbourhood_group_cleansed      0
host_acceptance_rate            281
host_response_time              473
host_identity_verified            0
host_has_profile_pic              0
number_of_reviews                 0
instant_bookable                  0
host_is_superhost                 0
dtype: int64

In [73]:
df_use['host_response_time'].value_counts()

within an hour        281
within a few hours    109
within a day          104
a few days or more     33
Name: host_response_time, dtype: int64

In [74]:
df_use['price'] = df_use.price.apply(lambda x: re.sub("[^0-9]", "", x[1:-3])).astype(int)

<ipython-input-74-0a47eacf652c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_use['price'] = df_use.price.apply(lambda x: re.sub("[^0-9]", "", x[1:-3])).astype(int)


In [78]:
df_use.head(10)

,price,review_scores_rating,availability_90,neighbourhood_group_cleansed,host_acceptance_rate,host_response_time,host_identity_verified,host_has_profile_pic,number_of_reviews,instant_bookable,host_is_superhost
6724,27,60.0,0,Mitte,0,within a day,No,Yes,2,No,No
6921,22,100.0,0,Neukölln,None,NaN,Yes,Yes,7,No,No
9006,50,100.0,0,Neukölln,None,NaN,Yes,Yes,8,No,No
15451,26,100.0,0,Mitte,100,NaN,Yes,Yes,2,Yes,No
14850,56,100.0,89,Mitte,61,within a day,Yes,Yes,2,No,No
15903,100,97.0,10,Treptow - Köpenick,100,within an hour,Yes,Yes,6,No,No
10835,110,100.0,29,Charlottenburg-Wilm.,100,NaN,Yes,Yes,24,Yes,Yes
3059,40,80.0,0,Tempelhof - Schöneberg,None,NaN,No,Yes,2,No,No
5770,19,90.0,0,Charlottenburg-Wilm.,72,NaN,Yes,Yes,43,No,No
13936,70,100.0,89,Friedrichshain-Kreuzberg,55,a few days or more,Yes,Yes,6,No,No


In [76]:
df_use.loc[df_use['host_acceptance_rate'].isna(), "host_acceptance_rate"] = '999%'
df_use['host_acceptance_rate'] = df_use['host_acceptance_rate'].apply(lambda x: x[:-1])
df_use.loc[df_use['host_acceptance_rate']=='999', "host_acceptance_rate"] = None

/Users/xinwen/opt/anaconda3/envs/python3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
<ipython-input-76-504e4e784ef5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_use['host_acceptance_rate'] = df_use['host_acceptance_rate'].apply(lambda x: x[:-1])


In [77]:
df_use['host_identity_verified'] = df_use['host_identity_verified'].replace({'t': 'Yes', 'f': 'No'})
df_use['host_has_profile_pic'] = df_use['host_has_profile_pic'].replace({'t': 'Yes', 'f': 'No'})
df_use['instant_bookable'] = df_use['instant_bookable'].replace({'t': 'Yes', 'f': 'No'})
df_use['host_is_superhost'] = df_use['host_is_superhost'].replace({'t': 'Yes', 'f': 'No'})

<ipython-input-77-b55141fac902>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_use['host_identity_verified'] = df_use['host_identity_verified'].replace({'t': 'Yes', 'f': 'No'})
<ipython-input-77-b55141fac902>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_use['host_has_profile_pic'] = df_use['host_has_profile_pic'].replace({'t': 'Yes', 'f': 'No'})
<ipython-input-77-b55141fac902>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [79]:
df_use.to_csv('availability90.csv', index=False)

## Review Word Cloud

In [80]:
df_reviews = pd.read_csv('reviews.csv')

In [82]:
df_reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,1944,7126992,2013-09-07,8207524,Mirko,I want to thank Laura&Emiliano for their hospi...
1,1944,7428447,2013-09-19,3021574,Rafiee,Very convenient and very quiet. You will stay...
2,1944,8455250,2013-10-31,5875429,Grzegorz,I've spent 2 nights at place of Laura and Emil...
3,1944,11105498,2014-03-20,5361252,Ngọc Thúy,The reservation was canceled 2 days before arr...
4,1944,15920963,2014-07-18,6659444,Nathalie,Laura est très sympathique et l'appartement fa...


In [122]:
df_reviews_sub = df_reviews.sample(n=1000, random_state=42)

In [123]:
df_reviews_sub = df_reviews_sub.merge(df_listing[["id", "price", "review_scores_rating", "availability_90"]], left_on='listing_id', right_on='id')



In [124]:
df_reviews_sub.head()

,listing_id,id_x,date,reviewer_id,reviewer_name,comments,id_y,price,review_scores_rating,availability_90
0,21474656,341132945,2018-10-26,34118864,Nenad,"Nice apartment, in the the very centre of Berl...",21474656,$65.00,94.0,51
1,901837,414666137,2019-02-20,33931273,Nora,Excellent location for a trip to Berlin! Close...,901837,$135.00,98.0,86
2,901837,123039860,2016-12-27,30598296,Cheryl,the apartment was exceptionally decorated and ...,901837,$135.00,98.0,86
3,1845815,498646502,2019-07-30,267402584,Leon,"Very cosy place, good building with lift, firs...",1845815,$51.00,86.0,82
4,14546260,153600374,2017-05-21,53815400,Silvia,Nathalie e Charlie sono persone davvero specia...,14546260,$65.00,98.0,0


In [125]:
df_reviews_sub['price'] = df_reviews_sub.price.apply(lambda x: re.sub("[^0-9]", "", x[1:-3])).astype(int)

In [143]:
#price > 100, price < 30
# review >= 85, review <= 80
d = {}
for row in df_reviews_sub[df_reviews_sub['review_scores_rating'] < 85].iterrows():
    row = row[1]
    comment = row['comments'].lower()
    comment = re.sub("[^a-z]", " ", comment).split(' ')
    for word in comment:
        d[word] = d.get(word, 0) + 1

d

{'nice': 6,
 'place': 9,
 'that': 5,
 'is': 12,
 'very': 12,
 'well': 2,
 'located': 4,
 '': 457,
 'would': 3,
 'definitely': 2,
 'recommend': 2,
 'for': 6,
 'a': 16,
 'trip': 1,
 'to': 14,
 'berlin': 6,
 'ok': 1,
 'appartement': 3,
 'sehr': 5,
 'gut': 2,
 'gelegen': 1,
 'ruhig': 1,
 'im': 1,
 'hinterhof': 1,
 'mit': 2,
 'garten': 1,
 'carmen': 1,
 'ist': 1,
 'freundlich': 2,
 'wir': 3,
 'haben': 1,
 'tolle': 2,
 'tage': 1,
 'in': 14,
 'erlebt': 1,
 'kommen': 1,
 'bestimmt': 1,
 'wieder': 3,
 'f': 2,
 'r': 2,
 'spontane': 1,
 'und': 5,
 'kurze': 1,
 'aufenthalte': 1,
 'zu': 2,
 'empfehlen': 1,
 'agr': 1,
 'able': 1,
 'pour': 3,
 'visiter': 1,
 'la': 3,
 'ville': 1,
 'sans': 1,
 'y': 1,
 'passer': 1,
 'trop': 1,
 'de': 6,
 'temps': 1,
 'car': 1,
 'peu': 1,
 'clair': 1,
 'avec': 1,
 'am': 1,
 'nagement': 1,
 'fonctionnel': 1,
 'adutes': 1,
 'accessible': 1,
 'depuis': 1,
 'l': 1,
 'roport': 1,
 'tegel': 1,
 'par': 2,
 'le': 1,
 'bus': 2,
 'txl': 1,
 'min': 1,
 'et': 4,
 'autres': 2,
 'li

In [144]:
stop_list = ['this', 'that', 'with', 'from', 'of', 'there', 'were', 'have', 'also', 'appartement']
d = {key:d[key] for key in d if d[key] > 3 and d[key] < 21 and len(key) > 3 and key not in stop_list}

d

{'nice': 6,
 'place': 9,
 'very': 12,
 'located': 4,
 'berlin': 6,
 'sehr': 5,
 'good': 4,
 'city': 5,
 'close': 4,
 'great': 4,
 'location': 4}

In [145]:
with open("review_below_85.json", "w") as outfile:  
    json.dump(d, outfile)

In [142]:
d = {}
for row in df_reviews_sub[df_reviews_sub['availability_90'] >= 60].iterrows():
    row = row[1]
    try:
        comment = row['comments'].lower()
        comment = re.sub("[^a-z]", " ", comment).split(' ')
        for word in comment:
            d[word] = d.get(word, 0) + 1
    except:
        print(row['comments'])
        pass

In [140]:
stop_list = ['this', 'that', 'with', 'from', 'of', 'there', 'were', 'have', 'also', 'appartement']
d = {key:d[key] for key in d if d[key] > 10 and d[key] < 200 and len(key) > 3 and key not in stop_list}

d

{'excellent': 20,
 'location': 90,
 'trip': 11,
 'berlin': 118,
 'close': 56,
 'public': 26,
 'which': 23,
 'host': 67,
 'next': 16,
 'visit': 12,
 'apartment': 125,
 'even': 18,
 'than': 17,
 'great': 154,
 'quiet': 28,
 'area': 31,
 'well': 33,
 'stay': 87,
 'will': 15,
 'time': 18,
 'place': 124,
 'good': 71,
 'building': 12,
 'easy': 33,
 'access': 12,
 'nice': 88,
 'neighborhood': 16,
 'haben': 26,
 'aufenthalt': 14,
 'sehr': 101,
 'central': 17,
 'enough': 11,
 'eine': 23,
 'zimmer': 18,
 'auch': 12,
 'gerne': 16,
 'wieder': 25,
 'lage': 28,
 'stayed': 11,
 'really': 52,
 'friendly': 18,
 'highly': 27,
 'recommend': 56,
 'staying': 12,
 'here': 17,
 'wonderful': 14,
 'einen': 11,
 'enjoyed': 25,
 'room': 47,
 'again': 27,
 'about': 18,
 'transport': 21,
 'around': 24,
 'city': 29,
 'small': 11,
 'comfortable': 35,
 'shower': 14,
 'away': 14,
 'local': 11,
 'would': 54,
 'super': 57,
 'unterkunft': 15,
 'spacious': 24,
 'kind': 14,
 'messages': 11,
 'home': 21,
 'only': 21,
 'bath

In [141]:
with open("availability90_above_60.json", "w") as outfile:  
    json.dump(d, outfile)